<a href="https://colab.research.google.com/github/mannmoshe/deep-learning/blob/main/medicine_imaging_project/medicine_imaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import shutil
import pathlib
import imageio
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt

In [3]:
tf.__version__

'2.12.0'

create directories

In [4]:
!mkdir train
!mkdir validation
!mkdir test

create subdirectories for each class

In [5]:
!mkdir train/01
!mkdir train/02
!mkdir train/03

!mkdir validation/01
!mkdir validation/02
!mkdir validation/03

!mkdir test/01
!mkdir test/02
!mkdir test/03

In [6]:
class1_path = '/content/gdrive/MyDrive/M.sc Data Mining/Medicine imaging/data/01/'
class2_path = '/content/gdrive/MyDrive/M.sc Data Mining/Medicine imaging/data/02/'
class3_path = '/content/gdrive/MyDrive/M.sc Data Mining/Medicine imaging/data/03/'

In [7]:
class1_files = os.listdir(class1_path)
class2_files = os.listdir(class2_path)
class3_files = os.listdir(class3_path)

In [8]:
class1_files_sorted = sorted([x for x in class1_files])
class2_files_sorted = sorted([x for x in class2_files])
class3_files_sorted = sorted([x for x in class3_files])

In [9]:
class1_files_sorted[:5]

['01.jpeg', '010.jpeg', '011.jpeg', '012.jpeg', '013.jpeg']

In [10]:
print(len(class1_files_sorted), len(class2_files_sorted), len(class3_files_sorted))

70 70 111


split class 1 to train, validation and test

In [11]:
for f in class1_files_sorted[:35]:
  src = os.path.join(class1_path, f)
  dst = os.path.join('/content/train/01/', f)
  shutil.copyfile(src, dst)

In [12]:
for f in class1_files_sorted[35:53]:
  src = os.path.join(class1_path, f)
  dst = os.path.join('/content/validation/01/', f)
  shutil.copyfile(src, dst)

In [13]:
for f in class1_files_sorted[53:]:
  src = os.path.join(class1_path, f)
  dst = os.path.join('/content/test/01/', f)
  shutil.copyfile(src, dst)

split class 2 to train, validation and test

In [14]:
for f in class2_files_sorted[:35]:
  src = os.path.join(class2_path, f)
  dst = os.path.join('/content/train/02/', f)
  shutil.copyfile(src, dst)

In [15]:
for f in class2_files_sorted[35:53]:
  src = os.path.join(class2_path, f)
  dst = os.path.join('/content/validation/02/', f)
  shutil.copyfile(src, dst)

In [16]:
for f in class2_files_sorted[53:]:
  src = os.path.join(class2_path, f)
  dst = os.path.join('/content/test/02/', f)
  shutil.copyfile(src, dst)

split class 3 to train, validation and test

In [17]:
for f in class3_files_sorted[:35]:
  src = os.path.join(class3_path, f)
  dst = os.path.join('/content/train/03/', f)
  shutil.copyfile(src, dst)

In [18]:
for f in class3_files_sorted[35:53]:
  src = os.path.join(class3_path, f)
  dst = os.path.join('/content/validation/03/', f)
  shutil.copyfile(src, dst)

In [19]:
for f in class3_files_sorted[53:70]:
  src = os.path.join(class3_path, f)
  dst = os.path.join('/content/test/03/', f)
  shutil.copyfile(src, dst)

In [20]:
print(len(os.listdir('/content/train/01/')), len(os.listdir('/content/train/02/')), len(os.listdir('/content/train/03/')))

35 35 35


In [21]:
print(len(os.listdir('/content/validation/01/')), len(os.listdir('/content/validation/02/')), len(os.listdir('/content/validation/03/')))

18 18 18


In [22]:
print(len(os.listdir('/content/test/01/')), len(os.listdir('/content/test/02/')), len(os.listdir('/content/test/03/')))

17 17 17


Build network

In [23]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [25]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [26]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        '/content/train/',
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/content/validation/',
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

Found 105 images belonging to 3 classes.
Found 54 images belonging to 3 classes.


In [27]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20, 3)


In [62]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

<ipython-input-62-a7acfc8093a4>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30


/usr/local/lib/python3.9/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


InvalidArgumentError: ignored